<table align="center" width=100%>
    <tr>
        <td width="15%">
            <img src="in_class.png">
        </td>
        <td>
            <div align="center">
                <font color="#21618C" size=8px>
                    <b> Inclass - Lab <br>(Week 1)
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

### About the data set (Heart disease data)

The dataset contains information about patients. The aim is to find whether or not a patient has cardiovascular disease based on the several factors from the patient profile. <br> Attribute information: 

**male:** Gender of patient: male(1) or female(0) 

**age:** Age of patient

**education:** Education level of patient: 1 = Some High School; 2 = High School or GED; 3 = Some College or Vocational School; 4 = college

**currentSmoker:** Whether the patient is a smoker now: Yes(1) or No(0)

**cigsPerDay:** Average number of cigarettes smoked by patient per day

**BPMeds:** Whether the patient is on blood pressure medications: Yes(1) or No(0)

**prevalentStroke:** If the patient has a history of heart stroke: Yes(1) or No(0)

**prevalentHyp:** If the patient has a history of hypertension: Yes(1) or No(0)

**diabetes:**  If the patient has a diabetes: Yes(1) or No(0)

**totChol:** Average cholesterol levels in mg/dL

**sysBP:** Average systolic blood pressure of patient

**diaBP:** Average diastolic blood pressure of patient

**BMI:** BMI of patient

**heartRate:** Avergae heartrate of patient

**glucose:** Average blood glucose levels of patient

**CVD:** Does the patient have cardiovascular disease: Yes(1) or No(0)

## Table of Content

1. **[Logistic Regression Model](#logreg)**
2. **[Model Evaluation Metrics](#mod_eval)**
3. **[Performance Evaluation Metrics](#perf_eval)**
4. **[Determining optimal threshold](#opt)**

**Import the required libraries**

In [1]:
# import 'Pandas' 
import pandas as pd 

# import 'Numpy' 
import numpy as np

# import subpackage of Matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# import 'Seaborn' 
import seaborn as sns

# to suppress warnings 
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = None
 
# to display the float values upto 6 decimal places     
pd.options.display.float_format = '{:.6f}'.format

# import train-test split 
from sklearn.model_selection import train_test_split

# import various functions from statsmodels
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler 

# import various functions from sklearn 
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

# import 'is_string_dtype' to check the string type 
from pandas.api.types import is_string_dtype

# import function to perform feature selection
from sklearn.feature_selection import RFE

#### Load the csv file

In [2]:
# type your code here
df_heart=pd.read_csv('Heart_disease.csv')
df_heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,CVD
0,1,39,4.000000,0,0.000000,0.000000,0,0,0,195.000000,106.000000,70.000000,26.970000,80.000000,77.000000,0
1,0,46,2.000000,0,0.000000,0.000000,0,0,0,250.000000,121.000000,81.000000,28.730000,95.000000,76.000000,0
2,1,48,1.000000,1,20.000000,0.000000,0,0,0,245.000000,127.500000,80.000000,25.340000,75.000000,70.000000,0
3,0,61,3.000000,1,30.000000,0.000000,0,1,0,225.000000,150.000000,95.000000,28.580000,65.000000,103.000000,1
4,0,46,3.000000,1,23.000000,0.000000,0,0,0,285.000000,130.000000,84.000000,23.100000,85.000000,85.000000,0


In [15]:
df_heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3989 entries, 0 to 4239
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             3989 non-null   object 
 1   age              3989 non-null   int64  
 2   education        3989 non-null   object 
 3   currentSmoker    3989 non-null   object 
 4   cigsPerDay       3989 non-null   float64
 5   BPMeds           3989 non-null   float64
 6   prevalentStroke  3989 non-null   object 
 7   prevalentHyp     3989 non-null   object 
 8   diabetes         3989 non-null   object 
 9   totChol          3989 non-null   float64
 10  sysBP            3989 non-null   float64
 11  diaBP            3989 non-null   float64
 12  BMI              3989 non-null   float64
 13  heartRate        3989 non-null   float64
 14  glucose          3658 non-null   float64
 15  CVD              3989 non-null   object 
 16  BPMeda           3989 non-null   object 
dtypes: float64(8),

In [3]:
# convert the columns that are wrongly specified as numeric to object type 

# convert the column male to object type
df_heart['male'] = df_heart['male'].astype(object)

# convert the column education to object type
df_heart['education'] = df_heart['education'].astype(object)

# convert the column currentSmoker to object type
df_heart['currentSmoker'] = df_heart['currentSmoker'].astype(object)

# convert the column BPMeds to object type
df_heart['BPMeds'] = df_heart['BPMeds'].astype(object)

# convert the column prevalentStroke to object type
df_heart['prevalentStroke'] = df_heart['prevalentStroke'].astype(object)

# convert the column prevalentHyp to object type
df_heart['prevalentHyp'] = df_heart['prevalentHyp'].astype(object)

# convert the column diabetes to object type
df_heart['diabetes'] = df_heart['diabetes'].astype(object)

In [5]:
# isnull().sum(): calculate the null values in each column for dataset
# sort_values(ascending=False): sort the values in descending order of values
total = df_heart.isnull().sum().sort_values(ascending=False)

# isnull().sum(): calculate the null values in each column for dataset
# isnull().count(): calculate the count values in each column for dataset
# sort_values(ascending=False): sort the values in descending order of values
percent = (df_heart.isnull().sum()/df_heart.isnull().count()).sort_values(ascending=False)

# concat(): append dataframes
# axis=1: append by column
# keys: specify column name
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

# display dataframe of missing values
missing_data

,Total,Percent
glucose,388,0.091509
education,105,0.024764
BPMeds,53,0.012500
totChol,50,0.011792
cigsPerDay,29,0.006840
BMI,19,0.004481
heartRate,1,0.000236
CVD,0,0.000000
diaBP,0,0.000000
sysBP,0,0.000000


Data Preprocessing

In [10]:
# dropna(): drop null values 
# subset: provide selected list of columns
df_heart = df_heart.dropna(subset = ['education', 'BPMeds', 'totChol','cigsPerDay','BMI','heartRate'])
df_heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,CVD,BPMeda
0,1,39,4.000000,0,0.000000,0.000000,0,0,0,195.000000,106.000000,70.000000,26.970000,80.000000,77.000000,0,0.000000
1,0,46,2.000000,0,0.000000,0.000000,0,0,0,250.000000,121.000000,81.000000,28.730000,95.000000,76.000000,0,0.000000
2,1,48,1.000000,1,20.000000,0.000000,0,0,0,245.000000,127.500000,80.000000,25.340000,75.000000,70.000000,0,0.000000
3,0,61,3.000000,1,30.000000,0.000000,0,1,0,225.000000,150.000000,95.000000,28.580000,65.000000,103.000000,1,0.000000
4,0,46,3.000000,1,23.000000,0.000000,0,0,0,285.000000,130.000000,84.000000,23.100000,85.000000,85.000000,0,0.000000


In [6]:
df_heart['glucose'] = df_heart['glucose'].fillna(df_heart['glucose'].median())

### Let's begin with some hands-on practice exercises

<a id="logreg"> </a>
## 1. Logistic regression model

## We shall use the heart disease dataset

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>1. Are all the classes of target variable 'CVD' fairly represented by records in the considered dataset ?</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [11]:
# Presence of classes in each column
df_target=df_heart['CVD']
print(df_target.value_counts())
print(df_target.count())

0    3596
1     644
Name: CVD, dtype: int64
4240


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>2. Predict whether or not a patient will have cardiovascular disease based on the information about blood pressure of the patient. Columns related to blood pressure are diaBP, sysBP and BPMeds.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [30]:
# type your code here
# fillna(): fill null values  with the median value
# value_counts().index[0]: indicates the mode for a categorical variable
df_heart.head()
df_num=df_heart.iloc[ :, [10,11]]
# scale all the numeric independent variables
#print(df_num.head())
num_scaled=df_num.apply(lambda rec:rec-rec.mean()/rec.std(),axis=0)
#print(num_scaled.head())
X = pd.DataFrame(num_scaled, columns = df_num.columns)

# add the categorical variable to train set dataframe
# convert the categorical variable to dummy variable
# get_dummies(): converts the variable to categorical variable
# drop_first=True: indicates n-1 dummy enoding; if set to false indicates one-hot encoding
X['BPMeds'] = pd.get_dummies(df_heart["BPMeds"], drop_first=True)

# set target variable
y = df_heart.CVD
X=sm.add_constant(X)
X.head()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.2)
logreg=sm.Logit(y_train,X_train).fit()
print(logreg.summary())

Optimization terminated successfully.
         Current function value: 0.403100
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    CVD   No. Observations:                 3392
Model:                          Logit   Df Residuals:                     3388
Method:                           MLE   Df Model:                            3
Date:                Sun, 25 Apr 2021   Pseudo R-squ.:                 0.05233
Time:                        13:09:53   Log-Likelihood:                -1367.3
converged:                       True   LL-Null:                       -1442.8
Covariance Type:            nonrobust   LLR p-value:                 1.601e-32
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.5233      0.319    -14.163      0.000      -5.149      -3.897
sysBP          0.0281      0.

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>3. Predict whether or not a patient has cardiovascular disease using the categorical variables in the dataset. How does a unit change in each feature influence the odds of a patient having a cardiocascular disease?</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [47]:
# type your code here
#df_heart.info()
df_cat = df_heart.select_dtypes(include="object")
df_cat
X = pd.get_dummies(df_cat, drop_first=True)
X.head()
y = df_heart.CVD
X = sm.add_constant(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.2)
logreg=sm.Logit(y_train,X_train).fit()
logreg.summary()
np.exp(logreg.params)

Optimization terminated successfully.
         Current function value: 0.399087
         Iterations 6


const               0.099330
male_1              1.748116
education_2.0       0.641914
education_3.0       0.789188
education_4.0       0.768474
currentSmoker_1     1.218203
BPMeds_1.0          1.815638
prevalentStroke_1   1.395520
prevalentHyp_1      2.436307
diabetes_1          2.738225
dtype: float64

In [52]:
y_pred_prob = logreg.predict(X)
y_pred = [ 0 if x < 0.5 else 1 for x in y_pred_prob]
cm = confusion_matrix(y_pred_prob, y_pred)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>4. Predit if a patient has cardiovascular disease based on whether or not the patient has history of hypertension. Calculate the odds ratio.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<a id="mod_eval"> </a>
## 2. Model Evaluation Metrics

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>5. Build a full model to predict if a patient will have a cardiovascular disease. Find the value of Mcfadden's R<sup>2</sup>.
                </font>
            </div>
        </td>
    </tr>
</table>

In [21]:
# type your code here
cat=df_heart.select_dtypes('object')
x_cat=pd.get_dummies(cat,drop_first=True)
x_num=df_heart.drop(['male', 'education', 'currentSmoker', 'BPMeds', 'prevalentStroke',
       'prevalentHyp', 'diabetes','CVD'],axis=1)

x=pd.concat([x_cat,x_num],axis=1)

xc=sm.add_constant(x)
y=df_heart['CVD']

X_train, X_test, y_train, y_test = train_test_split(xc, y, test_size=0.2, random_state=42)

full_model=sm.Logit(y_train,X_train).fit()
full_model.summary()


MissingDataError: exog contains inf or nans

In [22]:
df_numeric = df_heart.select_dtypes(include=np.number).drop(["CVD"],axis=1)
num_scaled=df_numeric.apply(lambda rec:rec-rec.mean()/rec.std(),axis=0)
df_num = pd.DataFrame(num_scaled, columns = df_num.columns)
df_categorical = df_heart.select_dtypes(include="object")
df_cat = pd.get_dummies(df_categorical, drop_first=True)
X = pd.concat([df_num,df_cat],axis=1)
y = df_heart.CVD
X = sm.add_constant(X)
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, random_state = 1, test_size = 0.2)
# build the model on train data 
# use fit() to fit the logistic regression model
logreg_full = sm.Logit(y_train_full, X_train_full).fit()

# let 'y_pred_prob' be the predicted values of y
y_pred_prob_full = logreg_full.predict(X_test_full)

# print the summary of the model
print(logreg_full.summary())


NameError: name 'df_num' is not defined

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>6. Find the significant variables in the full model when all the variables are considered in prediction of whether or not a patient has cardiovascular disease.
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# summary

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>7. How do the coefficients of each feature form the dataset impact the odds of a patient having a cardiovascular disease?
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here



<a id="perf_eval"> </a>
## 3. Performance evaluation metrics

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                        <b>8. For the full model, calculate the accuracy manually using the confusion matrix. Consider 0.5 as the probability threshold.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here
y_pred_prob = logreg_full.predict(X_test_full)
y_pred = [ 0 if x < 0.5 else 1 for x in y_pred_prob]


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>9.  Calculate value of kappa for the full model built in question 5. Consider threshold value as 0.18</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here
# let 'y_pred_prob' be the predicted values of y
y_pred_prob = logreg_full.predict(X_test_full)

# convert probabilities to 0 and 1 using 'if_else'
y_pred = [ 0 if x < 0.18 else 1 for x in y_pred_prob]

# compute the kappa value
kappa = cohen_kappa_score(y_test_full, y_pred)

# print the kappa value
print('kappa value:',kappa)


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>10. Identify the features from the dataset that are involved in multicollinearity. After that, split the updated data using train_test_split.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>11. Use the data obtained from Q10 and identify 5 features that contribute most in the prediction of target variable.<br></b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>12. Build a model using the features obtained in question 11. For the model find: 
                        <ol>                       
                         <li>Accuracy</li>
                        <li>F<sub>1</sub> score</li>
                        </ol>
                        </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here
# pass the actual and predicted target values to the f1_score()
print("F1 score:",f1_score(y_test_full, y_pred))


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>13. Compare the full model in question 5 and the model built in question 12 using their ROC curves. </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>14. Build a logistic regression model using information about heart rate of the patients. Compute the AUC score.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>15. Calculate the cross entropy for the model built in question 14.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here
# convert probabilities to 0 and 1 using 'if_else'
y_pred = [ 0 if x < 0.5 else 1 for x in y_pred_prob]

# pass the actual and predicted values to log_loss()
print("Log loss:",log_loss(y_test_rate,y_pred))


<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>16. Compare the model built in question 14 to the full model built in question 5.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>17. What is the cross entropy for the full model? Use the full model in Q5.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>18. Predict whether a patient has cardiovascular disease based on smoking habits of the patient. For the model find the following:<br> <ol type = "a">
                        <li> Precision </li>
                        <li> Recall</li>   
                        <li> F<sub>1</sub> score </li>
                        </ol>
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [24]:
# type your code here
# consider the numeric independent variable
df_num = df_heart.cigsPerDay

# scale all the numeric independent variables
num_scaled=df_numeric.apply(lambda rec:rec-rec.mean()/rec.std(),axis=0)

# create a dataframe of scaled numerical variables
# pass the required column names to the parameter 'columns'
df_num = pd.DataFrame(num_scaled, columns = ['cigsPerDay'] )

# convert the categorical variable to dummy variable
# get_dummies(): converts the variable to categorical variable
# drop_first=True: indicates n-1 dummy enoding; if set to false indicate one-hot encoding
df_cat = pd.get_dummies(df_heart.currentSmoker, drop_first=True)


In [25]:
# concatenate the scaled numerical and dummy variables
# axis: specifies whether to drop labels from index or columns; use 1 for columns and 0 for index
X = pd.concat([df_num,df_cat],axis=1)

# consider the dependent variable
y = df_heart.CVD

# add a constant column to the dataframe
# while using the 'Logit' method in the Statsmodels library, the method do not consider the intercept by default
# we can add the intercept to the set of independent variables using 'add_constant()'
X = sm.add_constant(X)

# split data into train subset and test subset
# set 'random_state' to generate the same dataset each time you run the code 
# 'test_size' returns the proportion of data to be included in the testing set
X_train_smoke, X_test_smoke, y_train_smoke, y_test_smoke = train_test_split(X, y, random_state = 1, test_size = 0.2)


In [26]:
# build the model on train data 
# use fit() to fit the logistic regression model
logreg = sm.Logit(y_train_smoke, X_train_smoke).fit()

# let 'y_pred_prob' be the predicted values of y
y_pred_prob = logreg.predict(X_test_smoke)

# convert probabilities to 0 and 1 using 'if_else'
y_pred = [ 0 if x < 0.5 else 1 for x in y_pred_prob]

# print classification report for the predictions
print(classification_report(y_test_smoke, y_pred))


MissingDataError: exog contains inf or nans

<a id="opt"> </a>
## 4. Determining optimal threshold

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>19. Obtain the optimal value threshold for the full model using the Youden's index.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [27]:
# type your code here
# build the model on train data 
# use fit() to fit the logistic regression model
logreg = sm.Logit(y_train_smoke, X_train_smoke).fit()

# let 'y_pred_prob' be the predicted values of y
y_pred_prob = logreg.predict(X_test_smoke)

# convert probabilities to 0 and 1 using 'if_else'
y_pred = [ 0 if x < 0.5 else 1 for x in y_pred_prob]

# print classification report for the predictions
print(classification_report(y_test_smoke, y_pred))


MissingDataError: exog contains inf or nans

In [28]:
# build the model on train data 
# use fit() to fit the logistic regression model
logreg_full = sm.Logit(y_train_full, X_train_full).fit()

# let 'y_pred_prob' be the predicted values of y
y_pred_prob_full = logreg_full.predict(X_test_full)

# print the summary of the model
print(logreg_full.summary())
# use the full model in Q5
# let 'y_pred_prob' be the predicted values of y
y_pred_prob = logreg_full.predict(X_test_full)

# obtain the values for false positive rate, true positive rate and threshold
# pass the actual target values and predicted probabilities to the function
fpr, tpr, thresholds = roc_curve(y_test_full, y_pred_prob)

# create a dataframe to store the values for false positive rate, true positive rate and threshold
youdens_table = pd.DataFrame({'TPR': tpr,
                             'FPR': fpr,
                             'Threshold': thresholds})

# calculate the difference between TPR and FPR for each threshold and store the values in a new column 'Difference'
youdens_table['Difference'] = youdens_table.TPR - youdens_table.FPR

# sort the dataframe based on the values of difference 
# 'ascending = False' sorts the data in descending order
# 'reset_index' resets the index of the dataframe
# 'drop = True' drops the previous index
youdens_table = youdens_table.sort_values('Difference', ascending = False).reset_index(drop = True)

# print the first five observations
youdens_table.head()



NameError: name 'y_train_full' is not defined

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>20. Consider the costs of false negatives and false positives as 3 and 1.3 respectively to obtain the optimal cut-off probability for which the total cost will be minimum.</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [1]:
# type your code here